Notebook for annotations using the geneformer result

In [ ]:
import scanpy as sc
import numpy as np
from tqdm.notebook import tqdm
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scvi
import glob

In [ ]:
data_dir = "/mnt/sata2/Analysis_Alex_2/perturb1/SI*"

Using the path to the time course, get the Class > Type > Subtype Heirarchy

In [ ]:
types = sc.read("timecourse.h5ad")
types_ = pd.crosstab(types.obs["Type"], types.obs["Subtype"])
class_ = pd.crosstab(types.obs["Class"], types.obs["Type"])
immuno_ = pd.crosstab(types.obs["Immunocentric_Type"], types.obs["Subtype"])

Create Class > Type > Subtype Heirarchy in replicates based off the GeneFormer result

In [ ]:
for path in glob.glob(data_dir):
    ad_sp = sc.read(os.path.join(path, "adatas", "06_geneformer_celltypes.h5ad"))
    # ad_sp.obsm['X_mde'] = scvi.model.utils.mde(ad_sp.obsm['X_scVI_replicates'])

    ad_sp.obs["Subtype"] = ad_sp.obs["celltype_predicted"].values

    type_dictionary = {}
    for i in ad_sp.obs["celltype_predicted"].cat.categories:
        type_dictionary[i] = types_.index.values[np.where(types_[i].values > 0)[0]][0]
    all_types = []
    for k in ad_sp.obs["celltype_predicted"].values:
        all_types.append(type_dictionary.get(k))
    ad_sp.obs["Type"] = all_types

    itype_dictionary = {}
    for i in ad_sp.obs["celltype_predicted"].cat.categories:
        try:
            itype_dictionary[i] = immuno_.index.values[
                np.where(immuno_[i].values > 0)[0]
            ][0]
        except:
            itype_dictionary[i] = "None"
    all_itypes = []
    for k in ad_sp.obs["celltype_predicted"].values:
        all_itypes.append(itype_dictionary.get(k))
    ad_sp.obs["Immunocentric_Type"] = all_itypes

    class_dictionary = {}
    for i in np.unique(ad_sp.obs["Type"].values):
        class_dictionary[i] = class_.index.values[np.where(class_[i].values > 0)[0]][0]
    all_classes = []
    for k in ad_sp.obs["Type"].values:
        all_classes.append(class_dictionary.get(k))
    ad_sp.obs["Class"] = all_classes

    sc.pp.neighbors(ad_sp, use_rep="X_pca")
    sc.tl.leiden(ad_sp, resolution=2)
    ad_sp.write(os.path.join(path, "adatas", "08_full_celltypes_and_leiden.h5ad"))
    # sc.pl.embedding(ad_sp, basis='mde', color=['celltype_predicted', 'Type', 'Class', 'Immunocentric_Type'])